# Feature Engineering for Equipment Failure and Predictive Maintenance Problems

For more information on the topic, please see the following article.

https://medium.com/swlh/machine-learning-for-equipment-failure-prediction-and-predictive-maintenance-pm-e72b1ce42da1


When it comes to equipment failure,  how you transform your variables is usually the most critical step.  In data science, there are almost always multiple ways to approach a problem.  Feature engineering is no different.  My background is in econometrics and classical statistics, so I tend to examine these problems from that perspective.  

In this article, I walk through the type of transformations that I use when facing a panel data problem.  Clearly, I do not present an exhaustive list of feature engineering. That would be impossible.   I do, however, establish a framework to show what is possible.

Feature engineering and variable transformation are synonyms as far as I am concerned.  I will use these terms interchangeably.  The same goes for panel data, cross-sectional time series, and repeated measures, which represent the same thing in my vernacular. When you have multiple entities measured over numerous periods of time, you have panel data/cross-sectional time-series/repeated measures. 

The data we use in this exercise is 100% fake, and it belongs to me.  I created it from scratch.  

Note that I designed this tutorial to run in Watson Studio.  If you have issues running in a different environment, please create a free account on the IBM cloud and try it there.

https://www.ibm.com/cloud/free

## Table of Contents

1. [Getting Setup](#setup1)<br>
 
2. [Data Exploration](#explore)<br>

3. [Dummy Variables](#3)<br>
 
4. [Mean encoding](#4)<br>
5. [Static Transformations](#5)<br>
6. [Interactive Transformations](#6)<br>
7. [Time Series Transformation](#7)<br>
    7.1 [Time Series Examination](#71)<br>
    7.2 [Buffer each entity](#72)<br>
    7.3 [Create Rolling Summaries](#73)<br>
    7.4 [Changes in the status quo](#74)<br>
    7.5 [Fun with lags](#75)<br>
    7.6 [Differences](#76)<br>
    7.7 [Changes over changes](#77)<br>
    7.8 [Variance within a Signal](#78)<br>
11. [Conclusions](#8)<br>

### 1.0 Getting Set-Up <a id="setup1"></a>

 Install all of the relevant Python Libraries

In [ ]:

#!pip install imbalanced-learn -- upgrade
!pip install plotly --upgrade
!pip install chart-studio --upgrade




Import required libraries

In [ ]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly as plotly
import pandas as pd
import numpy as np



Import the data from GitHub.

In [ ]:
#Remove the data if you run this notebook more than once
!rm fe_equipment_failure_data_1.csv

In [ ]:
#import first half from github
!wget https://raw.githubusercontent.com/shadgriffin/feature_engineering_equipment_failure/main/fe_equipment_failure_data_1.csv

In [ ]:
# Convert csv to pandas dataframe
pd_data_1 = pd.read_csv("fe_equipment_failure_data_1.csv", sep=",", header=0)

In [ ]:
#Remove the data if you run this notebook more than once
!rm fe_equipment_failure_data_2.csv

In [ ]:
#Import the second half from github
!wget https://raw.githubusercontent.com/shadgriffin/feature_engineering_equipment_failure/main/fe_equipment_failure_data_2.csv

In [ ]:
# convert to pandas dataframe
pd_data_2 = pd.read_csv("fe_equipment_failure_data_2.csv", sep=",", header=0)

In [ ]:
#concatenate the two data files into one dataframe
pd_data=pd.concat([pd_data_1, pd_data_2])



### 2.0 Data Exporation <a id="explore"></a>

In [ ]:
pd_data.head()

Now that we have the data imported into a Jupiter Notebook, we can explore it. Here is metadata explaining all of the fields in the data set.

ID — ID field that represents a specific machine.

DATE — The date of the observation.

MANUFACTURER — the company that manufactured the equipment in question.

S15 — A Sensor Value.

EQUIPMENT_FAILURE — A ‘1’ means that the equipment failed. A ‘0’ means the equipment did not fail.


Now we will walk through the data.

Examine the number of rows and columns.  The data has 307,751 rows and 5 columns.

In [ ]:

pd_data.shape

There are 421 machines in the data set.

In [ ]:

xxxx = pd.DataFrame(pd_data.groupby(['ID']).agg(['count']))
xxxx.shape

there are 731 unique dates in the data set.

In [ ]:

xxxx = pd.DataFrame(pd_data.groupby(['DATE']).agg(['count']))
xxxx.shape

We have 731 unique dates.  So if we have 421 machines and 731 unique dates, we should have 307,751 total records.  Based on the .shape command, we have one record per machine per date value.  There are no duplicates in the data frame.



And to triple confirm, remove all duplicates and count the rows again.

In [ ]:
df_failure_thingy=pd_data
df_failure_thingy=df_failure_thingy.drop_duplicates(subset=['ID','DATE'])
df_failure_thingy.shape


We can also explore the data with descriptive statistics.

In [ ]:

pd_data.describe()

### 3.0 Dummy Variables <a id="3"></a>

Oldie but Goodie, dummy variables are the bread and butter of variable transformation.  Dummy variables allow you to convert character fields into numerical values.   Some algorithms can handle categorical text fields, but many cannot. Dummy variables bridge the gap from text to number by creating a binary field for each unique value in the categorical field.  

For example, the field MANUFACTURER in our current data set has ten distinct values. 

In [ ]:
xxxx = pd.DataFrame(pd_data.groupby(['MANUFACTURER'])['ID'].agg('count'))
xxxx

We will convert the single field into ten individual binary variables, where a '1' means the value appears in MANUFACTURER and a '0' means it does not.

In [ ]:
df_dv = pd.get_dummies(pd_data['MANUFACTURER'])

pd_data = pd.concat([pd_data, df_dv], axis=1)
pd_data.head()

So, why do you need to represent a categorical field as a series of numerical fields? That's a good question, and, of course, there is not a single answer. It depends on the algorithm you use. 

For example, with multivariate statistical or parametric algorithms, dummy variables are necessary. Parametric algorithms would include things like logistic regression and linear regression (there are many others). While multivariate statistics is complex and has no doubt frustrated many undergraduates over the years, at its core, it is in many ways straightforward. Just like a computer is really about reading and writing '1's and '0's, most statistical models come from means, variances and covariances. The mean is an average, but you probably know that. The variance is the standard deviation squared. The covariance is the shared variance between two variables.

What do all these have in common? 

They are all dependent on the calculation of a mean. 

If you can not calculate a mean on a variable, you can not use it in a multivariate statistical model. Plain and simple. You can calculate an average on the binary fields we created above; therefore, you can use them in a regression model. You cannot calculate a mean on MANUFACTURER; thus, you cannot use it in a regression model without transforming it first.




### 4.0 Mean encoding <a id="4"></a>

Mean encoding is probably the most potent data transformation available. It is also the most dangerous and tricky to use.  With mean encoding, you summarize the dependent variable for a categorical variable and then use that summary as an independent variable in your model. 

I wrote an entire article on the subject.  Before attempting to use mean encoding, I would study this article in detail.

https://towardsdatascience.com/leveraging-value-from-postal-codes-naics-codes-area-codes-and-other-funky-arse-categorical-be9ce75b6d5a

### 5.0 Static Transformations <a id="5"></a>

Static transformations would include feature engineering that does not look backward in time. These are pretty common and well documented. Some of these transformations include the natural log, inverse, and square root. 

These transformations can be beneficial when capturing what we economists refer to as increasing and diminishing returns to scale. For example, adding resources increases production in a production environment but often with a diminishing return. In other words, the first employee you hire will be more productive than the 100th employee you hire.

Also, you will see diminishing returns to scale in consumer acquisition problems. It is typically cheaper to acquire the first ten customers than the customers 1,000 through 1,010. Most marketers don't think of it that way. They use the term "low hanging fruit," which has a lot more "sizzle" than decreasing returns to scale.
These types of transformations are also helpful when the relationship between the independent and dependent variable is quadratic. For example, the relationship between price and revenue is fundamentally quadratic. There is an optimal price. Pricing below or above the optimal is, well, sub-optimal.
https://medium.com/the-shadow/a-theoretical-and-practical-review-of-elasticity-and-pricing-strategy-a42b1cc09dfe

Note that these types of transformations are less useful in tree-based models.  Tree algorithms typically pick up non-linear relationships without a transformation.


In [ ]:
#add 1 to S15 5o prevent because the natural log of 0 is undefined.
pd_data['LN_S15']=np.log((pd_data['S15']+1))

pd_data['INV_S15']=1/(pd_data['S15'])

pd_data['SQRT_S15']=np.sqrt((pd_data['S15'])+1)

Now, let's take a look at these transformations on a single entitiy.

In [ ]:
d=pd_data[pd_data['ID']==100002]

x1=d['DATE']
y1=d['S15']
z1=d['INV_S15']
z2=d['LN_S15']
z3=d['SQRT_S15']

trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=z1,
    name='Inverse',
    yaxis='y2'
)
trace3 = go.Scatter(
    x=x1,
    y=z2,
    name='Natural Log',
    yaxis='y1'
)
trace4 = go.Scatter(
    x=x1,
    y=z3,
    name='Square Root',
    yaxis='y1'
)
data = [trace1, trace2,trace3, trace4]
layout = go.Layout(
    title='S15 and Static Transformations',
    yaxis=dict(title='S15'
    ),
        yaxis2=dict(
        title='Inverse',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

### 6.0 Interactive Transformations <a id="6"></a>

Interactive transformations involve creating a new variable from the product or ratio of two existing variables.  

An excellent example of a meaningful interactive transformation comes from predicting an individual's risk of diabetes.  Two variables that predict the onset of diabetes are weight and height, in combination.  

Someone who weighs 250lbs is probably at risk for diabetes if they are five feet tall, but not at risk if they are six feet six inches tall. In other words, height and weight are predictive separately but are much more predictive when combined.  The combination of height and weight is so predictive of disease (including diabetes) it has a name, the Body Mass Index (BMI). 

Another way to think of interactive variables is in the context of classical regression.  A dummy variable usually is static over time, right?  Typically, it doesn't change as you move from one day to the next.  If a Manufacturer is Y on Monday, it will be the same on Tuesday.  

Because they are static, when you include a dummy variable in a linear regression, it changes the intercept for records that contain a value of '1'.  

When you multiply a dummy variable times a continuous variable, you create a new interactive variable.  Doing so makes a second slope parameter.  In our current example, if you multiply Y times S15, you would distinguish the impact of time on S15 when Y is '1' and when it is '0'.

Of course, things are not so straightforward with some modern machine learning algorithms, but understanding what an interactive term adds to the model often helps one to know which variables to create.

I would also add that interactive dummy variables are less useful in tree-based machine learning algorithms.  Tree algorithms will find these interactions on their own.  In fact, many tree models were explicitly designed to find interactions.  

CHAID, for example, is an acronym for "Chi-Square Automatic Interaction Detector." https://en.wikipedia.org/wiki/Chi-square_automatic_interaction_detection

In [ ]:
#use two variables to create an interactive term
pd_data['S15_Y']=(pd_data['S15']*pd_data['Y'])
pd_data['S15_Z']=(pd_data['S15']*pd_data['Z'])

### 7.0 Time Series Transformation <a id="7"></a>

Up to this point, we have discussed feature engineering methods that are not time-dependent.  Now, we will look at ways to use previous values of a time-dependent variable to create new features in our data set.

#### 7.1 Time Series Examination<a id="71"></a>

Like I mentioned earlier, I typically approach these problems from the perspective of classical statistics and econometrics. The first step I go through is to examine the time plot of the series.  With Panel Data, we have multiple entities measured over time.  In other words, we have more than one time series.  Given this, I will randomly pick a few of the entities to examine or aggregate the entities by date.  Whatever makes sense.  For this use case, I just selected a few entities at random.
  

Again, I am not trying to do an exhaustive examination here.  I am just trying to get a feel of the data's personality.





In [ ]:
d=pd_data[pd_data['ID']==100002]

x1=d['DATE']
y1=d['S15']
z1=d['EQUIPMENT_FAILURE']

trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=z1,
    name='Failure',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='S15 Vs Failure',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='Failure',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

The series bounces around quite a bit but looks to have multiple means over time.  Except for March, April, and a few days in January, the series's mean is entirely different in 2016 than it was in 2015. 



In [ ]:
d=pd_data[pd_data['ID']==100024]

x1=d['DATE']
y1=d['S15']
z1=d['EQUIPMENT_FAILURE']

trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=z1,
    name='Failure',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='S15 Vs Failure',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='Failure',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

Ok, this looks different than the first sample.  The mean and variance are reasonably consistent except for a few spikes. Towards the end of 2016, the series drifts upward.

In [ ]:
d=pd_data[pd_data['ID']==100131]

x1=d['DATE']
y1=d['S15']
z1=d['EQUIPMENT_FAILURE']

trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=z1,
    name='Failure',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='S15 Vs Failure',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='Failure',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

With this series, you see a mean change around June 2015.  The variance is relatively consistent, but there are spikes in values that occur periodically after the failure on May 9th, 2015.

So what does all this mean?  Honestly, not much. However, I have a better idea of what the series is about and can make a few deductions.   

The series has occasional spikes.  Transformations (like running means and lagged values) that capture abrupt changes over time could be helpful.  

One thing I don't see in the data is seasonality.  Clear seasonality dictates the length of your transformation window.  For example, if the series has a weekly pattern, ensure that the transformations include at least seven days.  Likewise, if you see a daily pattern, include at least 24 hours in your transformation window.


In the rest of this notebook, I will describe multiple ways to transform this time-dependent variable (S15).  Which one works the best for you, and your problem depends on the nature of the series.  I believe it is essential to understand what you are transforming before you transform it.



#### 7.2 Buffer each entity <a id="72"></a>

We will use different running summaries and lags to transform the sensor value.  When doing this, we must ensure that entities do not "bleed" into one another.  For example, if we take a 30-day average, the first 29 days of the current entity will include values from the previous entity unless we explicitly prohibit them.  This is specific to panel data. 

In the next few steps, I will create a few variables that make it easy to keep this from happening.

In [ ]:

#transform date to a datetime variable
pd_data['DATE'] = pd.to_datetime(pd_data['DATE'])





Create a new field called “flipper” that indicates when the id changes as the data are sorted by ID and DATE in ascending order. We will use this in a few other transformations.

In [ ]:
pd_data=pd_data.sort_values(by=['ID','DATE'], ascending=[True, True])

pd_data['flipper'] = np.where((pd_data.ID != pd_data.ID.shift(1)), 1, 0)
pd_data.head()

The Feature window is the number of days we will look backward into a series. In this example, I used 14 as the feature window.  The value of the feature window is dependent on the context of your problem.  In this case, I have daily data for two years.  Given the density and length of the series, 14 days seems reasonable.  If, however, I had a series measured daily for 200 hundred years, 14 maybe a little short.  If my series is measured every second for 14 hours, 14 days will not work.  

You want to go back as far as necessary but not farther than necessary.  Clear as mud?  There are is much judgment here. I'd say experiment with different values and use common sense.  


The further you go back, the more data you will lose.   That's why it is crucial to keep the window as short as possible.

Let's say I have 14 days of data.  I only have one value if I use a feature window of twelve.  So, the longer the feature window, the more 'degrees of freedom' you lose in your model.  




As I mentioned earlier, I looked for a seasonal pattern in the series but did not see one.  If there were seasonality, my feature window would accommodate the seasonal pattern.  For example, a spike in the series every Sunday means you'd want the feature window to be at least seven days.  A monthly pattern requires at least a 30-day feature window.  An annual pattern requires at least a twelve-month window.   

In [ ]:
#define your feature window. This is the window by which we will aggregate our sensor values.
feature_window=14

Calculate the number of days from the first day a machine appears to the current day. This field will be called “TIME_SINCE_START” Also, create a variable called “too_soon.” When “too_soon” is equal to 1, we have less than 14 days (feature_window) of history for the machine.


In [ ]:
dfx=pd_data

In [ ]:
#Select the first record of each machine

starter=dfx[dfx['flipper'] == 1]

starter=starter[['DATE','ID']]

In [ ]:
#rename date to start_date
starter=starter.rename(index=str, columns={"DATE": "START_DATE"})

In [ ]:
#convert START_DATE to date
starter['START_DATE'] = pd.to_datetime(starter['START_DATE'])

In [ ]:
#Merge START_DATE to the original data set

dfx=dfx.sort_values(by=['ID', 'DATE'], ascending=[True, True])
starter=starter.sort_values(by=['ID'], ascending=[True])
dfx =dfx.merge(starter, on=['ID'], how='left')

In [ ]:
# calculate the number of days since the beginning of each well. 
dfx['C'] = dfx['DATE'] - dfx['START_DATE']
dfx['TIME_SINCE_START'] = dfx['C'] / np.timedelta64(1, 'D')
dfx=dfx.drop(columns=['C'])
dfx['too_soon'] = np.where((dfx.TIME_SINCE_START < feature_window) , 1, 0)

We added a variable called 'too_soon' to our data set. As we create variables, we will use this field to ensure that the summaries are unique to the machine (entity) in question.

#### 7.3 Create Rolling Summaries <a id="73"></a>

Rolling summaries are a prevalent type of transformation with panel data.  Rolling summaries "smooth" the series, eliminating the variance.  Many times, a rolling summary will make the nature of the series clearer and sometimes more predictive.  In other words, the fact that a value jumps from 5 to 7 to 5 to 7 over the last four hours may be less meaningful than it was, on average, six over the previous four hours.  Taking out the "noise" can often make variables more useful in machine learning models.

Create an object (shaggy) representing the variable for transformation.

In [ ]:

shaggy='S15'

Create a running average over various windows of time.

In [ ]:


for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+qq+'_'+'M'
    dfx[namer] = np.where((dfx.too_soon == 0),(dfx[shaggy].rolling(min_periods=1, window=feature_window).mean()) , dfx[shaggy])

 Notice how we added 14 new variables to our data frame.

In [ ]:
dfx.head()

Now let's create summaries for min and max.

Create a running max over various windows of time

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+qq+'_'+'X'
    dfx[namer] = np.where((dfx.too_soon == 0),(dfx[shaggy].rolling(min_periods=1, window=feature_window).max()) , dfx[shaggy])

Create a running min over various windows of time.

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+qq+'_'+'N'
    dfx[namer] = np.where((dfx.too_soon == 0),(dfx[shaggy].rolling(min_periods=1, window=feature_window).min()) , dfx[shaggy])

In [ ]:
dfx.head()

Let's take a look at the 12-day summaries visually.  Note I just picked the 12-day summaries for a visualization.  There is nothing magical about them.

In [ ]:
d=dfx[dfx['ID']==100131]

x1=d['DATE']
y1=d['S15']
y2=d['S15_12_M']
y3=d['S15_12_X']
y4=d['S15_12_N']

trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='12 Day Mean'
)
trace3 = go.Scatter(
    x=x1,
    y=y3,
    name='12 Day Max'
)
trace4 = go.Scatter(
    x=x1,
    y=y4,
    name='12 Day Min'
)
data = [trace1,trace2,trace3,trace4]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

#### 7.4 Changes in the status quo <a id="74"></a>

Another thing to look for is how the series is changing over time.  That is, if there is a jump in a sensor value, that may be meaningful.  The following transformations identify when the current value of the series differs from previous values. 

This signal transformation compares the current value to a running mean. If the current value is different from the previous values, that may be meaningful to a machine learning model.

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer='CHG'+'_'+shaggy+'_'+qq+'_'+'M'
    dfx[namer] = np.where((dfx[shaggy]> 0),(dfx[shaggy])/(dfx[shaggy+'_'+qq+'_'+'M']) , 0)

This transformation compares the current value to a running min.

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer='CHG'+'_'+shaggy+'_'+qq+'_'+'N'
    dfx[namer] = np.where((dfx[shaggy]> 0),(dfx[shaggy])/(dfx[shaggy+'_'+qq+'_'+'N']) , 0)

This transformation compares the current value to a running max.

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer='CHG'+'_'+shaggy+'_'+qq+'_'+'X'
    dfx[namer] = np.where((dfx[shaggy]> 0),(dfx[shaggy])/(dfx[shaggy+'_'+qq+'_'+'X']) , 0)

Now, let's examine these new signals visually.

In [ ]:
d=dfx[dfx['ID']==100131]

x1=d['DATE']
y1=d['S15']
y2=d['CHG_S15_12_M']
y3=d['CHG_S15_12_X']
y4=d['CHG_S15_12_N']

trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='value/12 Day Mean',
    yaxis='y2'
)
trace3 = go.Scatter(
    x=x1,
    y=y3,
    name='value/12 Day Max',
    yaxis='y2'
)
trace4 = go.Scatter(
    x=x1,
    y=y4,
    name='value/12 Day Min',
    yaxis='y2'
)


data = [trace1,trace2,trace3,trace4]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='Value over 12 Day Average',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

#### 7.5 Fun with lags <a id="75"></a>

Lagging variables is also sometimes a helpful transformation.  Lagging harkens back to the idea of an autoregressive model (the AR in ARIMA) in many ways.  Remember that with autoregression, today's value is a function of previous values.  Think of it like a weather forecast. An excellent way to predict the weather tomorrow is the weather today.  If it is 85 and Sunny today, tomorrow it probably won't be -12 and snowing.

With seasonal variation, lagging is extremely important.   In many retail businesses, 50% of sales come in December.  This means if you want to predict sales in December this year, you must know sales in December last year.  Lagging will do this.

Lag the sensor variable over various windows of time.  Very straightforward and often very useful.

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+'L'+qq
    dfx[namer] = np.where((dfx.too_soon == 0),dfx[shaggy].shift(x) , 0)

In [ ]:
d=dfx[dfx['ID']==100131]

x1=d['DATE']
y1=d['S15']
y2=d['S15_L12']


trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='12 Day Lag',
    yaxis='y2'
)


data = [trace1,trace2]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='12 Day Lag',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

#### 7.6 Differences <a id="76"></a>

Calculate the change between the current value and a lagged value over a window of time.  If you study classical time series, this is how you remove trends from time-dependent data.  Remember, the first step in building an ARIMA model is ensuring that the data is stationary.  Differencing (or integrating) is the "I" in ARIMA.  
 
Differences will remove trends and force the series closer to a constant mean.

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+'CHG'+'_'+qq
    dfx[namer] = np.where((dfx.too_soon == 0),(dfx[shaggy]-dfx[shaggy].shift(x)) , 0)

In [ ]:
d=dfx[dfx['ID']==100131]

x1=d['DATE']
y1=d['S15']
y2=d['S15_CHG_12']


trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='12 Day Difference',
    yaxis='y2'
)


data = [trace1,trace2]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='12 Day Lag',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

Notice how the difference has a constant mean of zero, where the original series bounces between 25 and 10.

Looking at the difference on a percentage basis can sometimes be helpful too.  

Calculate the percentage change between the current value and a lagged value over a window of time.

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+'CHG'+'_'+qq+'P'
    dfx[namer] = np.where(((dfx[shaggy].shift(x)) > 0),((dfx[shaggy]-dfx[shaggy].shift(x))/dfx[shaggy].shift(x)) , 0)
    dfx[namer] = np.where((dfx['too_soon']==0),(dfx[namer]) , 0)


In [ ]:
dfx.head()

In [ ]:
d=dfx[dfx['ID']==100131]

x1=d['DATE']
y1=d['S15']
y2=d['S15_CHG_12P']


trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='12 Day Percentage Change',
    yaxis='y2'
)


data = [trace1,trace2]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='12 Day Percentage Change',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

#### 7.7 Changes over changes <a id="77"></a>

This transformation is my favorite.  I often refer to it as a "Poor Man's Derivative." This transformation is beneficial when it is vital to understand how the signal is rising and falling with respect to time.  It is very much like a derivative in many ways. 

Calculate the current change by an average difference over a window of time.

(S15(t)-S15(t-1))/((S15(t)-S15(t-n))/n)

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+'D'+'_'+qq

    dfx[namer]=((dfx[shaggy]-dfx[shaggy].shift(1))/((dfx[shaggy]-dfx[shaggy].shift(x))/x))
    #replace nans
    dfx[namer] = dfx[namer].replace(np.nan, 0)
    #deal with buffer
    dfx[namer] = np.where((dfx['too_soon']==0),(dfx[namer]) , 0)
    #replace values that are inf
    dfx[namer]=np.where((dfx[shaggy]==dfx[shaggy].shift(x)),0,(dfx[namer]))


In [ ]:
d=dfx[dfx['ID']==100131]

x1=d['DATE']
y1=d['S15']
y2=d['S15_D_12']


trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='1 Day change over 12 Day change',
    yaxis='y2'
)


data = [trace1,trace2]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='1 Day change over 12 Day change',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

Sometimes it is helpful to look at the absolute value of the change.  

abs((S15(t)-S15(t-1))/((S15(t)-S15(t-n))/n))

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+'AD'+'_'+qq

    dfx[namer]=abs(((dfx[shaggy]-dfx[shaggy].shift(1))/(dfx[shaggy]-dfx[shaggy].shift(x))/x))
    #replace nans
    dfx[namer] = dfx[namer].replace(np.nan, 0)
    #deal with buffer
    dfx[namer] = np.where((dfx['too_soon']==0),(dfx[namer]) , 0)
    #replace infs
    dfx[namer]=np.where((dfx[shaggy]==dfx[shaggy].shift(x)),0,(dfx[namer]))

In [ ]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
d=dfx[dfx['ID']==100131]

x1=d['DATE']
y1=d['S15']
y2=d['S15_AD_12']


trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='Absolute value of 1 Day change over 12 Day change',
    yaxis='y2'
)


data = [trace1,trace2]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='Absolute value of 1 Day change over 12 Day change',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes-lines')

Sometimes it is helpful to look at the sign of the change.

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+'SD'+'_'+qq
    dfx[namer] = np.where((((dfx[shaggy]-dfx[shaggy].shift(1))/(dfx[shaggy]-dfx[shaggy].shift(x))/x))>0, 1 , -1)
    #replace nans
    dfx[namer] = dfx[namer].replace(np.nan, 0)
    #deal with buffer
    dfx[namer] = np.where((dfx['too_soon']==0),(dfx[namer]) , 0)
    #replace infs
    dfx[namer]=np.where((dfx[shaggy]==dfx[shaggy].shift(x)),0,(dfx[namer]))

In [ ]:
d=dfx[dfx['ID']==100131]

x1=d['DATE']
y1=d['S15']
y2=d['S15_SD_12']


trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='Sign of 1 Day change over 12 Day change',
    yaxis='y2',
    mode='markers'
)


data = [trace1,trace2]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='Sign of 1 Day change over 12 Day change',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes')

#### 7.8 Variance within a signal <a id="78"></a>

Often what is important is not the signal's value or the signal's change but the variance in the signal.  Many of the transformations previously mentioned will pick up on this, but here are a few other ways to capture this variance over time.

Compare the minimum and maximum values over a window of time.  


In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+'X_N'+'_'+qq

    dfx[namer] = np.where((dfx.too_soon == 0),(dfx[shaggy].rolling(min_periods=1, window=x).max())-(dfx[shaggy].rolling(min_periods=1, window=x).min()) , 0)

In [ ]:
d=dfx[dfx['ID']==100131]

x1=d['DATE']
y1=d['S15']
y2=d['S15_X_N_12']


trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='S15'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='Difference between Max and Min',
    yaxis='y2',
    mode='markers'
)


data = [trace1,trace2]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(
        title='S15'
    ),
    yaxis2=dict(
        title='Difference between Max and Min',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes')

Sometimes the best way to estimate the variance is to actually calculate the variance.  The following code estimates the variance and standard deviation of the signal over time.

In [ ]:
for x in range(1, (feature_window+1)):
    qq=str(x)
    namer=shaggy+'_'+'STD'+'_'+qq
    namer2=shaggy+'_'+'VAR'+'_'+qq
    dfx['x_bar']=dfx[shaggy].rolling(min_periods=1, window=x).mean()
    dfx['x']=dfx[shaggy]
    dfx['diff']=np.square(dfx['x']-dfx['x_bar'])
    dfx['runner']=dfx['diff'].rolling(min_periods=1,window=x).sum()
    dfx['S2']=dfx['runner']/x
    dfx['SD']=np.sqrt(dfx['S2']+1)
    dfx[namer] = dfx['SD']
    dfx[namer2]=dfx['S2']
    dfx.drop(columns=['x_bar', 'x','diff','S2','SD'])
    dfx[namer] = dfx[namer].replace(np.nan, 0)
    dfx[namer] = np.where((dfx['too_soon']==0),(dfx[namer]) , 0)
    dfx[namer2] = dfx[namer2].replace(np.nan, 0)
    dfx[namer2] = np.where((dfx['too_soon']==0),(dfx[namer2]) , 0)

    

In [ ]:
d=dfx[dfx['ID']==100131]

x1=d['DATE']
y1=d['S15_12_M']
y2=d['S15_STD_12']
y3=d['S15']


trace1 = go.Scatter(
    x=x1,
    y=y1,
    name='12 Day Moving Average',
    yaxis='y1'
)
trace2 = go.Scatter(
    x=x1,
    y=y2,
    name='12 Day Moving Standard Deviation (+1)',
    yaxis='y1',
    mode='markers'
)
trace3 = go.Scatter(
    x=x1,
    y=y3,
    name='S15',
    yaxis='y1'
)


data = [trace1,trace2,trace3]
layout = go.Layout(
    title='S15 and Summaries',
    yaxis=dict(

    ),
    yaxis2=dict(
        title='12 Day Moving Standard Deviation (+1)',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
        xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
))
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='shapes')

#### 8.0 Conclusions <a id="8"></a>

These data transformations are by no means an exhaustive list, but hopefully, it gives you some ideas on how to beef up your machine learning models with feature engineering.


Hopefully, this was helpful.

### Author





**Shad Griffin**, is a Certified Thought Leader and Data Scientist at IBM

<hr>
Copyright &copy; IBM Corp. 2021. This notebook and its source code are released under the terms of the MIT License.

